# POPPY Sim: MagAO-X focal plane propagation optimization - Ver: Science/NCP DM path
## Notes: 
- This is version 11.1 of the MagAO-X Zemax file, with NCP DM path information from Solidworks mode.
- This notebook focuses on the F/# focal plane generation to see if any distance corrections are needed. No aberration required.
- This notebook optimized focal plane distance correction for H-alpha wavelength, 512 sampling, and 4x oversample.

In [1]:
%matplotlib inline
#load modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
from astropy.io import fits
from  matplotlib.colors import LogNorm
from model_kit import magaoxFunctions as mf
#POPPY
import poppy
from poppy.poppy_core import PlaneType
print("POPPY Version: "+poppy.__version__)

POPPY Version: 0.9.2.dev11+g1887dd5


## Initiate the optical elements and variables
- The values for the Magellan telescope and MagAO-X are pulled from Oli Durney's Zemax file, version 11.1. The NCP DM path between OAP-4 and OAP-5-1 were pulled from the solidworks model designed by Cork. All MagAO-X elements (after F/11 focal plane) have their distances pulled from the Zemax file's Merit Function Editor (using RAG command).

### Optical elements not implemented currently:
- ADC1 and 2
- Lyot stop

In [19]:
d_m2_m3 + d_m3_wfsdichroic + d_wfsdichroic_peri1 + d_peri1_k1 + d_k1_k2 + d_k2_k3 + d_k3_f11fp

<Quantity 14.00233207 m>

In [16]:
testWavelength = 656.3e-9 * u.m # H-alpha
samp = 512  # original version at 512
oversamp = 0.25 # 4x

# physical radius values
M1_radius = 3.25 * u.m # This is the correct value to use.
M2_radius = 0.6695 * u.m # This is the correct value to use.
M3_radius = 0.439879 * u.m # Using largest value from various Zemax files
oap_radius = 0.0254 * u.m # 2 inch diameter OAP
flat_2in_radius = 0.025 * u.m # 50mm diameter
flat_1in_radius = 0.0125 * u.m # 25mm diameter
flat_0p5in_radius = 0.00625 * u.m # 12.5mm diameter
flat_19mm_radius = 0.0095 * u.m # 19mm diameter
dm_radius = 0.025 * u.m # 50mm diameter currently, subject to change if need to be larger

# Saved F/# correction values, calculated and copied from a previous run.
f11_delta = 2.6068628201869615e-05 * u.m
f16_delta = 0.00014377668927423315 * u.m
f57_delta = -7.080525876546062e-05 * u.m 
f69_delta = -4.621933338810891e-05 * u.m
f69sci_delta = 1.2789797395384994e-05 * u.m

# focal lengths for Magellan Telescope
fl_M1 = 8.12765 * u.m # Zemax definition
fl_M2 = 1.43141 * u.m # Zemax definition

# Focal lengths of OAPs
fl_oap0 = 0.143701 *u.m #0.14355 * u.m
fl_oap1 = 0.21088 * u.m
fl_oap2 = 0.30380 * u.m
fl_oap3 = 1.07160 * u.m
fl_oap4 = 0.51300 * u.m
fl_oap51 = 0.62100 * u.m
fl_oap52 = 0.62100 * u.m
fl_oap53 = 0.62100 * u.m

# propagation distances based on Zemax
# All values before F/11 is not present in Zemax MFE nor solo Magellan Zemax file, so going by distances listed.
d_m1_m2 = 9.72205 * u.m
d_m2_m3 = 9.02279 * u.m
d_m3_wfsdichroic = 4.849516 * u.m
d_wfsdichroic_peri1 = 0.040 * u.m
d_peri1_k1 = 0.020 * u.m
d_k1_k2 = 0.025 * u.m
d_k2_k3 = 0.025 * u.m
d_k3_f11fp = 0.020 * u.m + f11_delta 

# Inputted from Zemax MFE, all RAG values
d_f11fp_peri2 = 0.033204 * u.m
d_peri2_oap0 = 0.110497 * u.m 
d_oap0_woofer = 0.143701 * u.m #0.144693 * u.m
d_woofer_oap1 = 0.210726 * u.m
d_oap1_fold1 = 0.069174 * u.m
d_fold1_f16fp = 0.141562 * u.m + f16_delta
d_f16fp_oap2 = 0.303808 * u.m
d_oap2_tweeter = 0.303808 * u.m
d_tweeter_oap3 = 1.071599 * u.m # adds through front and back ADC
d_oap3_flat4 = 0.412515 *u.m
d_flat4_flat5 = 0.5737 * u.m
d_flat5_f57fp = 0.085449 * u.m + f57_delta
d_f57fp_oap4 = 0.513026 * u.m

# NCP DM path
d_oap4_NCPDM = 0.47666 * u.m
d_NCPDM_vapp = 0.03634 * u.m
d_vapp_flat6 = 0.19427 * u.m
d_flat6_oap51 = 0.42673 * u.m

# Rest of the optics train
d_oap51_fold2 = 0.471013 * u.m
d_fold2_f69fp = 0.150008 * u.m + f69_delta
d_f69fp_oap52 = 0.620969 * u.m
d_oap52_lyot = 0.621031 * u.m
d_lyot_fold3 = 0.231596 * u.m
d_fold3_oap53 = 0.389425 * u.m
d_oap53_fold4 = 0.401103 * u.m
d_fold4_f69sci = 0.219859 * u.m + f69sci_delta

# Primary and Secondary mirrors
M1 = poppy.QuadraticLens(fl_M1, name='M-1')
M2 = poppy.QuadraticLens(fl_M2, name='M-2')

# OAP mirrors
OAP0 = poppy.QuadraticLens(fl_oap0, name='OAP-0')
OAP1 = poppy.QuadraticLens(fl_oap1, name='OAP-1')
OAP2 = poppy.QuadraticLens(fl_oap2, name='OAP-2')
OAP3 = poppy.QuadraticLens(fl_oap3, name='OAP-3')
OAP4 = poppy.QuadraticLens(fl_oap4, name='OAP-4')
OAP51 = poppy.QuadraticLens(fl_oap51, name='OAP-5-1')
OAP52 = poppy.QuadraticLens(fl_oap52, name='OAP-5-2')
OAP53 = poppy.QuadraticLens(fl_oap53, name='OAP-5-3')

# load pupil mask
pupil = mf.surfFITS(file_loc='/home/jhen/XWCL/code/MagAOX/data/MagAOX_f11_pupil_{0}_unmasked.fits'.format(samp),
                 optic_type='trans', opdunit='none', name='MagAO-X Pupil (unmasked)')

In [17]:
# print out focal plane distances
(d_k3_f11fp.value, 
 d_fold1_f16fp.value, 
 d_flat5_f57fp.value, 
 d_fold2_f69fp.value, 
 d_fold4_f69sci.value)

(0.02002606862820187,
 0.14170577668927423,
 0.08537819474123454,
 0.1499617806666119,
 0.21987178979739538)

In [18]:
# print out delta correction
(f11_delta.value,
f16_delta.value,
f57_delta.value,
f69_delta.value,
f69sci_delta.value)

(2.6068628201869615e-05,
 0.00014377668927423315,
 -7.080525876546062e-05,
 -4.621933338810891e-05,
 1.2789797395384994e-05)

# Task 1: Focal Plane PSF generation

## Part 1: Build to F/11 focal plane

In [6]:
magaox = poppy.FresnelOpticalSystem(pupil_diameter=2*M1_radius, 
                                       npix=samp,
                                       beam_ratio=oversamp)
# Entrance Aperture
magaox.add_optic(poppy.CircularAperture(radius=M1_radius))

# Add Pupil
magaox.add_optic(pupil)

# Surface: Primary Mirror
magaox.add_optic(M1)
magaox.add_optic(poppy.CircularAperture(radius=M1_radius,name="M-1 aperture"))

# Surface: Secondary Mirror
magaox.add_optic(M2, distance=d_m1_m2)
magaox.add_optic(poppy.CircularAperture(radius=M2_radius,name="M-2 aperture"))

# Surface: Tertiary mirror
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="M-3"), 
                                            distance=d_m2_m3)
magaox.add_optic(poppy.CircularAperture(radius=M3_radius, name="M-3 aperture"))

# Surface: Periscope Mirror 1 (F-1)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-1"), 
                                            distance=d_m3_wfsdichroic+d_wfsdichroic_peri1)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="F-1 aperture"))

# Begin K-mirror setup
# Surface: K-1
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="K-1"), distance=d_peri1_k1)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="K-1 aperture"))

# Surface: K-2
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="K-2"), distance=d_k1_k2)
magaox.add_optic(poppy.CircularAperture(radius=flat_0p5in_radius, name="K-2 aperture"))

# Surface: K-3
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="K-3"), distance=d_k2_k3)
magaox.add_optic(poppy.CircularAperture(radius=flat_19mm_radius, name="K-3 aperture"))

# Surface: F/11 Focal Plane
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="F/11 focal plane (uncorrected)"), 
                                          distance=d_k3_f11fp)

### Need to calculate correction for F/11 focus point

In [10]:
# Focal plane correction for F/11
unfixed_f11fp_psf, unfixed_f11fp_wfs = magaox.calc_psf(wavelength=testWavelength.value, display_intermediates=False, return_intermediates=True)
f11_last_wave = unfixed_f11fp_wfs[-1]
f11_delta = f11_last_wave.z_w0 - f11_last_wave.z #f11_delta is the change in distance to compensate the gaussian beam waist.

# Incorporate the correction distance to see the F/11 PSF.
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F/11 focal plane (corrected)"), distance=f11_delta)

f11_delta.value

2.6068628201869615e-05

## Part 2: Build to ASM F/16 focal plane.
This is the start of the MagAO-X design.

In [7]:
# Surface: Periscope Mirror 2 (F-2)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-2"), distance=d_f11fp_peri2)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="F-2 aperture"))

# Surface: OAP-0 (O-0)
magaox.add_optic(OAP0, distance=d_peri2_oap0)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-0 aperture"))

# Surface: woofer DM mirror
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="woofer mirror"), distance=d_oap0_woofer)
magaox.add_optic(poppy.CircularAperture(radius=dm_radius, name="woofer mirror aperture"))

# Surface: OAP-1 (O-1)
magaox.add_optic(OAP1, distance=d_woofer_oap1)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-1 aperture"))

# Surface: Fold Mirror 1 (F-3)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-3"), distance=d_oap1_fold1)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="F-3 aperture"))

# Surface: ASM F/16 Focal Plane
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="ASM F/16 Focal Plane (uncorrected)"), distance=d_fold1_f16fp)

### Need to calculate correction for F/16 focus point

In [13]:
# Focal plane correction for F/16 ASM
unfixed_f16fp_psf, unfixed_f16fp_wfs = magaox.calc_psf(wavelength=testWavelength.value, 
                                                       display_intermediates=False, return_intermediates=True)
f16_last_wave = unfixed_f16fp_wfs[-1]
f16_delta = f16_last_wave.z_w0 - f16_last_wave.z #f16_delta is the change in distance to compensate the gaussian beam waist.

# Incorporate the correction distance to see the F/16 ASM PSF.
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="ASM F/16 focal plane (corrected)"), distance=f16_delta)

f16_delta.value

0.00014377668927423315

## Part 3: Build to F/57 Focal Plane

In [8]:
# Surface: OAP-2 (O-2)
magaox.add_optic(OAP2, distance=d_f16fp_oap2)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-2 aperture"))

# Surface: tweeter mirror
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="tweeter mirror"), distance=d_oap2_tweeter)
magaox.add_optic(poppy.CircularAperture(radius=dm_radius, name="tweeter mirror aperture"))

# SKIP: ADC, will go from tweeter to OAP-3

# Surface: OAP-3
magaox.add_optic(OAP3, distance=d_tweeter_oap3)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-3 aperture"))

# Surface: Flat mirror (F-4), Breadboard Mirror 1
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-4"), distance=d_oap3_flat4)
magaox.add_optic(poppy.CircularAperture(radius=flat_2in_radius, name="F-4 aperture"))

# Surface: Flat mirror (F-5), Breadboard Mirror 5
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-5"), distance=d_flat4_flat5)
magaox.add_optic(poppy.CircularAperture(radius=flat_2in_radius, name="F-5 aperture"))

# Surface: F/57 Focal Plane
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F/57 Focal Plane (uncorrected)"), distance=d_flat5_f57fp)

### Need to calculate correction for F/57 focus point

In [14]:
# Focal plane correction for F/57
unfixed_f57fp_psf, unfixed_f57fp_wfs = magaox.calc_psf(wavelength=testWavelength.value, 
                                                       display_intermediates=False, return_intermediates=True)
f57_last_wave = unfixed_f57fp_wfs[-1]
f57_delta = f57_last_wave.z_w0 - f57_last_wave.z 
#f57_delta is the change in distance to compensate the gaussian beam waist.

# Incorporate the correction distance to see the F/57 PSF.
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="F/57 focal plane (corrected)"), distance=f57_delta)

f57_delta.value

-7.080525876546062e-05

## Part 4: Build to F/69 Focal Plane

In [9]:
# Surface: OAP-4 (O-4)
magaox.add_optic(OAP4, distance=d_f57fp_oap4)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-4 aperture"))

# Surface: NCP DM
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="NCP DM"), 
                 distance=d_oap4_NCPDM)
magaox.add_optic(poppy.CircularAperture(radius=dm_radius, name="NCP DM aperture"))

# Surface: vAPP coronagraph (transmit through pupil plane)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="vAPP mask"), 
                 distance=d_NCPDM_vapp)

# Surface: Flat mirror (F-6), special L/10 silver mirror
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-6"), 
                 distance=d_vapp_flat6)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="F-6 aperture"))

# Surface: OAP-5 (O-5)
magaox.add_optic(OAP51, distance=d_flat6_oap51)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-5-1 aperture"))

# Surface: Fold Mirror 2 (F-7)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-7"), 
                 distance=d_oap51_fold2)
magaox.add_optic(poppy.CircularAperture(radius=flat_1in_radius, name="F-7 aperture"))

# Surface: F/69 Focal Plane
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="F/69 Focal Plane (uncorrected)"), distance=d_fold2_f69fp)

### Need to calculate correction for F/69 focus point

In [10]:
# Focal plane correction for F/69
unfixed_f69fp_psf, unfixed_f69fp_wfs = magaox.calc_psf(wavelength=testWavelength.value, 
                                                       display_intermediates=False, return_intermediates=True)
f69_last_wave = unfixed_f69fp_wfs[-1]
f69_delta = f69_last_wave.z_w0 - f69_last_wave.z 
#f69_delta is the change in distance to compensate the gaussian beam waist.

# Incorporate the correction distance to see the F/69 PSF.
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="F/69 focal plane (corrected)"), distance=f69_delta)

f69_delta.value

-4.621933338810891e-05

## Part 5: Build to Science F/69 Focal Plane

In [10]:
# Surface: OAP-6 (O-6)
magaox.add_optic(OAP52, distance=d_f69fp_oap52)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-5-2 aperture"))

# Surface: 9mm Lyot Stop
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="Lyot"), distance=d_oap52_lyot)

# Surface: Fold Mirror 3 (F-11)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-11"), distance=d_lyot_fold3)
magaox.add_optic(poppy.CircularAperture(radius=flat_2in_radius, name="F-11 aperture"))

# Surface: OAP-7
magaox.add_optic(OAP53, distance=d_fold3_oap53)
magaox.add_optic(poppy.CircularAperture(radius=oap_radius,name="OAP-5-3 aperture"))

# Surface: Fold Mirror 4 (F-12)
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F-12"), distance=d_oap53_fold4)
magaox.add_optic(poppy.CircularAperture(radius=flat_2in_radius, name="F-12 aperture"))

# Surface: Science F/69 Focal Plane
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, name="F/69 Science Focal Plane (uncorrected)"), distance=d_fold4_f69sci)

### Need to calculate correction for F/69 Science focus point

In [11]:
# Focal plane correction for F/69 Science
unfixed_f69scifp_psf, unfixed_f69scifp_wfs = magaox.calc_psf(wavelength=testWavelength.value, 
                                                            display_intermediates=False, 
                                                            return_intermediates=True)
f69sci_last_wave = unfixed_f69scifp_wfs[-1]
f69sci_delta = f69sci_last_wave.z_w0 - f69sci_last_wave.z 

# Incorporate the correction distance to see the F/69 PSF.
magaox.add_optic(poppy.ScalarTransmission(planetype=PlaneType.intermediate, 
                                          name="F/69 Science Focal Plane (corrected)"), 
                 distance=f69sci_delta)

f69sci_delta.value

1.2789797395384994e-05

### Find the vAPP pupil plane and calculate the radius for csv file

In [14]:
for n in range(0, len(unfixed_f69scifp_wfs)):
    loc = unfixed_f69scifp_wfs[n].location[5:12]
    pixscale = unfixed_f69scifp_wfs[n]._pixelscale_m
    diam = pixscale * samp*u.pix
    print('{0:2} - {1:7} - {2:10.10} - {3:.5}'.format(n, loc, pixscale, diam))

 0 -  Circle - 0.0126953125 m / pix - 6.5 m
 1 -  MagAO- - 0.0126953125 m / pix - 6.5 m
 2 -  M-1    - 0.0126953125 m / pix - 6.5 m
 3 -  M-1 ap - 0.0126953125 m / pix - 6.5 m
 4 -  M-2    - 0.002490437734 m / pix - 1.2751 m
 5 -  M-2 ap - 0.002490437734 m / pix - 1.2751 m
 6 -  M-3    - 0.0008856552898 m / pix - 0.45346 m
 7 -  M-3 ap - 0.0008856552898 m / pix - 0.45346 m
 8 -  F-1    - 1.601192696e-05 m / pix - 0.0081981 m
 9 -  F-1 ap - 1.601192696e-05 m / pix - 0.0081981 m
10 -  K-1    - 1.245475131e-05 m / pix - 0.0063768 m
11 -  K-1 ap - 1.245475131e-05 m / pix - 0.0063768 m
12 -  K-2    - 8.008281748e-06 m / pix - 0.0041002 m
13 -  K-2 ap - 8.008281748e-06 m / pix - 0.0041002 m
14 -  K-3    - 3.561812185e-06 m / pix - 0.0018236 m
15 -  K-3 ap - 3.561812185e-06 m / pix - 0.0018236 m
16 -  F/11 f - 1.801760812e-06 m / pix - 0.0009225 m
17 -  F-2    - 5.905623015e-06 m / pix - 0.0030237 m
18 -  F-2 ap - 5.905623015e-06 m / pix - 0.0030237 m
19 -  OAP-0  - 2.555848491e-05 m / pix - 

In [15]:
vAPP_diam = unfixed_f69scifp_wfs[43]._pixelscale_m * samp * u.pix
(vAPP_diam/2).value

0.004512827904881557